In [1]:
import pandas as pd
import numpy as np
from tpot import TPOTClassifier

c:\Users\Pkalanski\Anaconda3\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [2]:
train_df = pd.read_csv("data/train.csv")

In [3]:
from common import calculate_features

In [4]:
train_features_df = calculate_features(train_df)

In [5]:
X_train = train_features_df
y_train = train_df["Transported"]

In [6]:
tpot = TPOTClassifier(generations=10, population_size=100, verbosity=2, periodic_checkpoint_folder="tpot_checkpoint/", n_jobs=-1)
tpot.fit(X_train, y_train)
tpot.export('tpot_pipeline.py')

Imputing missing values in feature set
                                                                               
Generation 1 - Current best internal CV score: 0.7920194733822528
                                                                               
Generation 2 - Current best internal CV score: 0.7920194733822528
                                                                                 
Generation 3 - Current best internal CV score: 0.7920194733822528
                                                                                 
Generation 4 - Current best internal CV score: 0.7923646977781101
                                                                                 
Generation 5 - Current best internal CV score: 0.7923646977781101
                                                                                 
Generation 6 - Current best internal CV score: 0.793285693204896
                                                                              

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'], random_state=None)

imputer = SimpleImputer(strategy="median")
imputer.fit(training_features)
training_features = imputer.transform(training_features)
testing_features = imputer.transform(testing_features)

# Average CV score on the training set was: 0.793285693204896
exported_pipeline = RandomForestClassifier(bootstrap=True, criterion="entropy", max_features=0.6500000000000001, min_samples_leaf=18, min_samples_split=11, n_estimators=100)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import StandardScaler
from tpot.builtins import StackingEstimator, ZeroCount
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'], random_state=None)

imputer = SimpleImputer(strategy="median")
imputer.fit(training_features)
training_features = imputer.transform(training_features)
testing_features = imputer.transform(testing_features)

# Average CV score on the training set was: 0.7946651349829373
exported_pipeline = make_pipeline(
    ZeroCount(),
    StandardScaler(),
    StackingEstimator(estimator=SGDClassifier(alpha=0.001, eta0=0.01, fit_intercept=False, l1_ratio=1.0, learning_rate="invscaling", loss="squared_hinge", penalty="elasticnet", power_t=0.1)),
    XGBClassifier(learning_rate=0.5, max_depth=3, min_child_weight=15, n_estimators=100, n_jobs=1, subsample=0.7000000000000001, verbosity=0)
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)
